In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font size= "6" color="red">ch09 파일 io(입출력) 프로그래밍</font></b>
파일 : txt, pickle, csv, fson, hdf5(h5)
# 1절. txt파일에 데이터 저장하고 불러오기

In [32]:
f = open('data/ch09.txt', 'w')
print('쓰기 가능한지 여부 :', f.writable())

쓰기 가능한지 여부 : True


In [33]:
f.write('Hello\nWorld')
f.close() #close() 쓰기한 내용이 저장이 x

In [34]:
# mode : 'r' 또는 'rt' : text읽기 모드
      #  'rb' : 바이너리읽기 모드
      #  'w' 또는 'wt' : text쓰기 모드 (파일이 있으면 덮어씀, 파일이 없으면 파일 생성)
      #  'wb' : 바이너리쓰기 모드
      #  'a' 또는 'at' : text 추가 모드(파일이 있으면 append, 파일이 없으면 파일 생성)
      # 읽기모드에서는 파일이 없으면 예외/쓰기모드에서는 폴더가 없으면 예외
# encoding 
     # euc-kr (한글완성형) 믜x
     # cp949 (확장된 한글완성형) : open()함수 기본 encoding방식(win)
     # utf-8 (한글조합형): open()함수 기본 encoding방식(mac, linux), 주피터노트북 기본
with open('data/ch09.txt', 'w') as f:
    print('쓰기 가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

쓰기 가능한지 여부 : True


In [35]:
with open('data/ch09.txt', 'w') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('김길동, 33, 아무동9\n')
    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']
    for line in textlist:
        f.write(line)
    # 3방법
    f.writelines(textlist)

In [36]:
# 한줄씩 읽기
with open('data/ch09.txt', 'r') as f:
    line = f.readline()
    while line!='':
        print(line, end='')
        line = f.readline()

홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9


In [37]:
# 모든 줄을 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.readlines()
    print(lines)

['홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n', '홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n', '홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']


In [38]:
# 모두 읽기
with open('data/ch09.txt', 'r') as f:
    lines = f.read()
    print(lines)

홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9



# 2절. 피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 txt 데이터 불러오기

In [39]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age  = age
        self.email= email
        self.address = address
    def __str__(self):
        return "{}, {}, {}, {}".format(self.name,
                                      "성년" if self.age>18 else "미성년",
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
               }
    def __eq__(self, other):
        return self.name == other.name and \
                self.age == other.age and \
                self.email == other.email and \
                self.address == other.address

In [40]:
user1 = Member('홍',20,'a@a.com','신림동')
user2 = Member('홍',20,'a@a.com','신림동')
print(user1==user2)
print(user1.__eq__(user2)) # 같은 내용이면 True

True
True


In [41]:
# 형식이 있는 txt 파일 내용을 member list(피클로 저장), 딕셔너리 list(데이터프레임)로 저장
user_list = [] # member list
user_dict = [] # 딕셔너리 list
with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)
for line in lines:
    data = line.split(',')
#     print(data)
    name = data[0]
    age = int(data[1].strip()) # strip() : 좌우 공백(스페이스, \t, \n) 제거
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())

In [42]:
for user in user_list:
    print(user)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


## 2.2 피클링
- 객체 리스트(user_list) -> 피클파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)

In [43]:
# 피클링을 이용한 객체를 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [44]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

In [45]:
user_list == load_user_list

True

In [46]:
for idx in range(len(user_list)):
    print(idx, user)
    print(idx, load_user_list[idx])

0 신길동, 성년, shinkil@hong.com, 서울시 동작구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
1 신길동, 성년, shinkil@hong.com, 서울시 동작구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구


In [47]:
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
True
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
True
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
True


In [48]:
result = []
for user, load_user in zip(user_list, load_user_list):
    result.append(user==load_user)
all(result)

True

# 3절. CSV형식 파일 읽기/쓰기
- CSV파일 <-> 리스트데이터  3.1 / 3.2
- CSV파일 <-> 딕셔너리데이터 3.3 / 3.4

## 3.1 reader

In [49]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    result = list(reader)
print(result)

[['홍길동', '20', 'kildong@hong.com', '서울시 관악구'], ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구'], ['신길동', '30', 'sindong@hong.com', '서울시 동작구']]


In [50]:
# ""(따옴표)가 없는 데이터는 numeric으로
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, 
                        quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
print(dict_list)

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}, {'name': '김길동', 'age': 40, 'email': 'kimdong@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]


## 3.2 writer

In [51]:
user_list = [['홍길동', 20, 'kildong@hong.com', '서울시 관악구'],
             ['김길동', 40, 'kimdong@hong.com', '서울시 영등포구']]

In [52]:
import csv
with open('data/ch09_member1.write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(user_list)

In [53]:
import csv
with open('data/ch09_member1.write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(user_list)

## 3.3 DictReader

In [54]:
import csv
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
print(dict_list)
for row in dict_list:
    if row['job'] is None:
        print(row['name'], row['age'], row['email'], row['address'])
    else:
        print(row['name'], row['age'], row['email'], row['address'], row['job'])

[{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구', 'job': None}, {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구', 'job': '팀장'}, {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구', 'job': None}]
홍길동 20 h@h.com 서울시 관악구
신길동 40 s@h.com 서울시 영등포구 팀장
김길동 30 k@h.com 서울시 동작구


In [55]:
# header가 없는 csv파일('data/ch09_member1-cp.csv')을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f,
                                 fieldnames=['Name','Age','Email','Address','Job'])
    result=(list(dict_reader))
for row in result:
    print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


In [56]:
# header가 없는 csv파일('data/ch09_member1-cp.csv')을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f,
                                 fieldnames=['Name','Age','Email','Address'],
                                restkey='Job')
    result=(list(dict_reader))
for row in result:
    print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ['']}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': ['팀장']}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ['']}


## 3.4 DictWriter
- 딕셔너리 리스트 -> CSV파일

In [57]:
user1 = {'name': '홍길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'}
user2 = {'name': '신길동', 'age': 32, 'email': 'b@a.com', 'address': '신림동'}
user3 = {'name': '김길동', 'age': 42, 'email': 'c@a.com', 'address': '신림동'}
user_list = [user1,user2,user3]
fieldnames = list(user1.keys())

In [58]:
with open('data/ch09_member4.csv', 'w', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

## CSV <-> 데이터프레임

In [59]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv')
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@h.com,서울시 영등포구,팀장
2,김길동,30,k@h.com,서울시 동작구,NaN


In [60]:
type(member)

pandas.core.frame.DataFrame

# 4절. JSON 데이터 저장하고 불러오기(dump, load)
- 딕셔너리리스트 <-> JSON파일
- 객체리스트 <-> JSON 파일
## 4.1 dump(파일 출력)

In [61]:
data = [{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울'},
        {'name': '김길동', 'age': 30, 'email': 'kimdong@hong.com', 'address': '인천'}]

In [62]:
# ensure_ascii 매개변수
    # True : 비 아스키문은 유니코드 형태로 저장
    # False : 비 아스키문자 원래 형태로 저장
import json
with open('data/ch09_member.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data, #딕셔너리 리스트
             jsonfile,
             ensure_ascii=False,
             indent='  ')

In [63]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age  = age
        self.email= email
        self.address = address
    def __str__(self):
        return "{}, {}, {}, {}".format(self.name,
                                      self.age,
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
               }
    def __eq__(self, other):
        return self.name == other.name and \
                self.age == other.age and \
                self.email == other.email and \
                self.address == other.address

In [64]:
user1 = Member

In [65]:
member_list =[Member('홍길동',22, 'a@a.com', '서울'),
              Member('신길동',32, 'a@a.com', '서울'),
              Member('김길동',42, 'a@a.com', '서울')]

In [66]:
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list, # 객체 리스트
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict # 객체를 딕셔너리로 return 하는 인스턴스 함수
             )

## 4.2 load(파일 입력)
- json파일 -> 딕셔너리리스트(기본)
                   ↓
- json파일 ->   객체리스트

In [67]:
def as_member(dic):
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'), dic['age'], dic.get('email'), dic.get('address'))

In [68]:
member = as_member({'name':'김길동', 'age':20, 'email':'z@a.com', 'address':'관약'})
print(member)

김길동, 20, z@a.com, 관약


In [69]:
with open('data/ch09_member.json', 'r', encoding='utf-8') as f:
    load_data = json.load(f)
load_data

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울'},
 {'name': '김길동', 'age': 30, 'email': 'kimdong@hong.com', 'address': '인천'}]

In [70]:
with open('data/ch09_member1.json', 'r', encoding='utf-8') as f:
    load_member_list = json.load(f, object_hook=as_member)
for load_member in load_member_list:
    print(load_member)

홍길동, 22, a@a.com, 서울
신길동, 32, a@a.com, 서울
김길동, 42, a@a.com, 서울


## JSON -> 데이터프레임

In [71]:
import pandas as pd
pd.read_json('data/ch09_member1.json')

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


In [72]:
pd.read_json('data/ch09_member.json')

,name,age,email,address
0,홍길동,20,kildong@hong.com,서울
1,김길동,30,kimdong@hong.com,인천


# 5절. hdf5(h5)파일 쓰기/읽기
- h5py 사용
## 5.1 hdf5파일 쓰기
- 딕셔너리 리스트를 hdf5파일에 쓰기
- 데이터프레임을 hdf5파일에 쓰기

In [73]:
# seaborn : 시각화 패키지. 학습을 위한 데이터셋 다수
import seaborn as sns
iris_df = sns.load_dataset('iris')
iris_df[::50]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [74]:
iris_df.to_hdf('data/ch09_iris.hdf5', key='iris')

In [75]:
iris_dic = iris_df.to_dict()
type(iris_dic), type(iris_df)

(dict, pandas.core.frame.DataFrame)

In [76]:
iris_dic.keys()

dict_keys(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

In [77]:
for group, value in iris_dic.items():
    print('그룹명 :',group)
    for key, data in value.items():
        print('{}:{}'.format(str(key),data), end = '\t')
    print() # 개행

그룹명 : sepal_length
0:5.1	1:4.9	2:4.7	3:4.6	4:5.0	5:5.4	6:4.6	7:5.0	8:4.4	9:4.9	10:5.4	11:4.8	12:4.8	13:4.3	14:5.8	15:5.7	16:5.4	17:5.1	18:5.7	19:5.1	20:5.4	21:5.1	22:4.6	23:5.1	24:4.8	25:5.0	26:5.0	27:5.2	28:5.2	29:4.7	30:4.8	31:5.4	32:5.2	33:5.5	34:4.9	35:5.0	36:5.5	37:4.9	38:4.4	39:5.1	40:5.0	41:4.5	42:4.4	43:5.0	44:5.1	45:4.8	46:5.1	47:4.6	48:5.3	49:5.0	50:7.0	51:6.4	52:6.9	53:5.5	54:6.5	55:5.7	56:6.3	57:4.9	58:6.6	59:5.2	60:5.0	61:5.9	62:6.0	63:6.1	64:5.6	65:6.7	66:5.6	67:5.8	68:6.2	69:5.6	70:5.9	71:6.1	72:6.3	73:6.1	74:6.4	75:6.6	76:6.8	77:6.7	78:6.0	79:5.7	80:5.5	81:5.5	82:5.8	83:6.0	84:5.4	85:6.0	86:6.7	87:6.3	88:5.6	89:5.5	90:5.5	91:6.1	92:5.8	93:5.0	94:5.6	95:5.7	96:5.7	97:6.2	98:5.1	99:5.7	100:6.3	101:5.8	102:7.1	103:6.3	104:6.5	105:7.6	106:4.9	107:7.3	108:6.7	109:7.2	110:6.5	111:6.4	112:6.8	113:5.7	114:5.8	115:6.4	116:6.5	117:7.7	118:7.7	119:6.0	120:6.9	121:5.6	122:7.7	123:6.3	124:6.7	125:7.2	126:6.2	127:6.1	128:6.4	129:7.2	130:7.4	131:7.9	132:6.4	133:6.3	134:6.1	135:7.7	136

In [78]:
for group, value in iris_dic.items():
    print('그룹명 :',group)
#     print(value)

그룹명 : sepal_length
그룹명 : sepal_width
그룹명 : petal_length
그룹명 : petal_width
그룹명 : species


In [79]:
# 딕셔너리 리스트를 hdf파일로 쓰기
import h5py
with h5py.File('data/ch09_iris_dic.h5', 'w') as f:
    for group, value in iris_dic.items():
        grp = f.create_group(group)
        for key, data in value.items():
            grp.create_dataset(str(key), data==data)

## 5.2 hdf5 파일 읽기
- hdf5 파일을 딕셔너리 리스트로 읽기
- hdf5 파일을 데이터프레임으로 읽기

In [80]:
import pandas as pd
load_iris_df = pd.read_hdf('data/ch09_iris.hdf5', key='iris')
load_iris_df[::50]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


In [81]:
all(iris_df == load_iris_df) # 두 데이터프레임 변수의 모든 행과 열의 값이 같은지 여부

True

In [82]:
with h5py.File('data/ch09_iris_dic.h5', 'r') as f:
    for group, value in f.items():
        print('그룹 : ', group)
        for key, data in value.items():
            print('{}:{}'.format(key, data[()]), end='\t')
            # data[()] : h5 DataSet타입을 가져올 때

그룹 :  petal_length
0:[0.]	1:[0.]	10:[0.]	100:[0.]	101:[0.]	102:[0.]	103:[0.]	104:[0.]	105:[0.]	106:[0.]	107:[0.]	108:[0.]	109:[0.]	11:[0.]	110:[0.]	111:[0.]	112:[0.]	113:[0.]	114:[0.]	115:[0.]	116:[0.]	117:[0.]	118:[0.]	119:[0.]	12:[0.]	120:[0.]	121:[0.]	122:[0.]	123:[0.]	124:[0.]	125:[0.]	126:[0.]	127:[0.]	128:[0.]	129:[0.]	13:[0.]	130:[0.]	131:[0.]	132:[0.]	133:[0.]	134:[0.]	135:[0.]	136:[0.]	137:[0.]	138:[0.]	139:[0.]	14:[0.]	140:[0.]	141:[0.]	142:[0.]	143:[0.]	144:[0.]	145:[0.]	146:[0.]	147:[0.]	148:[0.]	149:[0.]	15:[0.]	16:[0.]	17:[0.]	18:[0.]	19:[0.]	2:[0.]	20:[0.]	21:[0.]	22:[0.]	23:[0.]	24:[0.]	25:[0.]	26:[0.]	27:[0.]	28:[0.]	29:[0.]	3:[0.]	30:[0.]	31:[0.]	32:[0.]	33:[0.]	34:[0.]	35:[0.]	36:[0.]	37:[0.]	38:[0.]	39:[0.]	4:[0.]	40:[0.]	41:[0.]	42:[0.]	43:[0.]	44:[0.]	45:[0.]	46:[0.]	47:[0.]	48:[0.]	49:[0.]	5:[0.]	50:[0.]	51:[0.]	52:[0.]	53:[0.]	54:[0.]	55:[0.]	56:[0.]	57:[0.]	58:[0.]	59:[0.]	6:[0.]	60:[0.]	61:[0.]	62:[0.]	63:[0.]	64:[0.]	65:[0.]	66:[0.]	67:[0.]	68:[0.]	69:[0.]	7:

In [83]:
load_iris_dic = {}
with h5py.File ('data/ch09_iris_dic.h5', 'r') as f:
    for group, value in f.items():
        temp = {}
        for key, data in value.items():
            temp[int(key)] = data[()]
        load_iris_dic[group] = temp
        
# load_iris_dic['s_l'] = {0:5.4, 1:4}
# load_iris_dic['s_w'] = {0:5.4, 1:4}
load_iris_dic.keys()

dict_keys(['petal_length', 'petal_width', 'sepal_length', 'sepal_width', 'species'])

# 6절. 연습문제 : 고객관리
- 프로그램 시작시 'data/data/ch09_customers.txt'(고객 데이터 백업용)의 내용을 load(객체list)
-                파일이 없을 경우 빈 파일 생성

- 'data/data/ch09_customers.txt'(고객 데이터 백업용)의 내용
    홍길동, 010-0000-0000, a@a.com, 30, 3, 까칠해
    홍길동, 010-0000-0000, a@a.com, 30, 3, 까칠해
    
## Customer 클래스 작성

In [84]:
class Customer:
    '고객데이터와 as_dic(), to_list_style(txt백업시), __str__()'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age  = age
        self.grade = grade
        self.etc = etc
    def as_dic(self):
        return {"name":self.name, 
                "phone":self.phone, 
                "email":self.email,
                "age":self.age, 
                "grade":self.grade, 
                "etc":self.etc}
    def to_list_style(self):
        # return "홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해"
#         return "{}, {}, {}, {}, {}, {}".format(self.name, self.phone, self.email,
#                                               self.age, self.grade, self.etc)
        temp = [self.name, 
                self.phone, 
                self.email, 
                str(self.age), 
                str(self.grade), 
                self.etc]
        return ', '.join(temp)
    def __str__(self):
        # return "  *** 홍길동, 010-9999-9999, a@a.com, 30, 까칠해"
        return "{:>5}\t{:3}\t{:13}\t{:15}\t{:3}\t{}".format('*'*self.grade,
                       self.name, self.phone, self.email, self.age, self.etc)

In [85]:
temp = ['홍길동','010-9999-9999', 'a@a.com', '30',' 3', '까칠해']
', '.join(temp)

'홍길동, 010-9999-9999, a@a.com, 30,  3, 까칠해'

In [86]:
def to_customer(row):
    '''
    row = "홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해"(txt파일 내용)을 매개변수로 받아
    Customer 객체로 return 
    '''
    data = row.split(',')
    name = data[0]
    phone = data[1].strip() # strip은 앞뒤 white space 제거
    email = data[2].strip()
    age = int(data[3].strip())
    grade = int(data[4].strip())
    etc = data[5].strip()
    return Customer(name, phone, email, age, grade, etc)

In [87]:
c1 = Customer("홍길동", "010-9999-9999", "a@a.com", 30, 3, "까칠해")
print(c1)
print(c1.as_dic())
print(c1.to_list_style())
c2 = to_customer("홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해")
print(c2)

  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
{'name': '홍길동', 'phone': '010-9999-9999', 'email': 'a@a.com', 'age': 30, 'grade': 3, 'etc': '까칠해'}
홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해


## 0. 처음 실행시 호출
- ch09_customers.txt(백업데이터)에서 customer_list return
- 파일이 없으면 빈 파일 생성, customer_list = [] return

In [88]:
def load_customers():
    customer_list = []
    try:
        with open('data/ch09_customers.txt', 'r', encoding='utf-8') as f:
            # txt 데이터 한줄씩 customer 객체로 받아 customer_list.append
            lines = f.readlines()
            for line in lines:
                customer = to_customer(line)
#                 print(customer)
                customer_list.append(customer)
    except:
        with open('data/ch09_customers.txt', 'w', encoding='utf-8') as f:
            print('초기화 파일을 생성했습니다.')
    return customer_list

In [89]:
customer_list = load_customers()
for customer in customer_list:
    print(customer)

  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	이길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	김길동	010-9999-9999	a@a.com        	 30	까칠해


In [90]:
def fn1_insert_customer_info():
    '''
    사용자로부터 name, phone, email, age, grade, etc를 입력받아 Customer형 객체 반환
    '''
#     import re
    name = input('이름 : ')
#     name_pattern = r'[가-힣]{2,}'
#     while not re.search(name_pattern, name):
#         print('이름을 제대로 입력하세요(한글 2글자 이상)')
    phone = input("전화번호 : ")
    email = input("이메일 : ")
    while True:
        try:
            age = int(input("나이 : "))
            if (age<0) | (age>130):
                raise Exception('나이를 정확히 입력하세요')
            break
        except:
            print('올바른 나이를 입력하세요')
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    customer = Customer(name, phone, email, age, grade, etc)
    return customer

In [91]:
# 1번 함수 테스트
customer = fn1_insert_customer_info()
customer_list.append(customer)
for customer in customer_list:
    print(customer)
    

이름 : 
전화번호 : 
이메일 : 
나이 : 4
고객등급(1~5) : 65
기타 정보 : 4
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	이길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	김길동	010-9999-9999	a@a.com        	 30	까칠해
*****	   	             	               	  4	4


## 2. 전체출력

In [92]:
def fn2_print_customers(customer_list):
    'customer_list를 출력(pdf 40page 스타일)'
    print('='*70)
    print('{:^70}'.format('고객 정보'))
    print('-'*70)
    print("{:>5}\t{:3}\t{:13}\t{:15}\t{:3}\t{}".format('GRADE', '이름', '전화',
                                                       '메일', '나이', '기타'))
    print('-'*70)
    for customer in customer_list:
        print(customer)

In [93]:
# 2번 기능 테스트
fn2_print_customers(customer_list)

                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	전화           	메일             	나이 	기타
----------------------------------------------------------------------
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	홍길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	신길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	이길동	010-9999-9999	a@a.com        	 30	까칠해
  ***	김길동	010-9999-9999	a@a.com        	 30	까칠해
*****	   	             	               	  4	4


## 3. 삭제

In [94]:
# 3. 삭제 (동명이인이 없을 경우 구현)
def fn3_delete_customer(customer_list):
    '''삭제하고자 하는 고객이름을 input으로 받아
    매개변수로 들어온 customer_list에서 삭제하고 "삭제했음/삭제못했음"을 메세지로 출력'''
    delete_name = input('삭제할 고객 이름은?')
    for customer in customer_list:
        if customer.name == delete_name:
            customer_list.remove(customer)
            print('{}님 데이터를 삭제하였습니다'.format(delete_name))
            break
    else:
        print('{}님 데이터는 존재하지 않습니다'.format(delete_name))

In [95]:
# 3. 삭제(동명이인이 있을 경우 해당 동명이인을 지울지 한사람 한사람 묻고 지우기 Y/N)
def fn3_delete_customer(customer_list):
    '''삭제하고자 하는 고객이름을 input으로 받아
    매개변수로 들어온 customer_list에서 삭제하고 "삭제했음/삭제못했음"을 메세지로 출력'''
    delete_name = input('삭제할 고객 이름은?')
    delete_idx = [] # 삭제할 인덱스 저장하는 용도
    for idx,customer in enumerate(customer_list):
        if customer.name == delete_name:
            delete_idx.append(idx)
    if delete_idx:
        for idx in delete_idx[::-1]: # [1,0]
            print(customer_list[idx], '지우겠습니까?', end='')
            answer = input('(Y/N)')
            if answer.upper == 'Y':
                print('요청하신 {}({})님 삭제 완료'.format(delete_name,
                                                 customer_list[idx].phone))
                del customer_list[idx]
    else:
        print('{}님 데이터가 존재하지 않습니다'.format(delete_name))

In [96]:
# 3. 테스트
customer_list = load_customers()
fn3_delete_customer(customer_list)

삭제할 고객 이름은?dasd
dasd님 데이터가 존재하지 않습니다


## 4. 이름찾기

In [102]:
# 4. 이름찾기
def fn4_search_customer(customer_list):
    search_name = input('이름')
    search_list = []
    for idx, customer in enumerate(customer_list):
        if customer.name == search_name:
            search_list.append(idx)
    if search_list:
        fn2_print_customers(search_list)
    else:
        print('{}는 없음'.format(search_name))
    

In [103]:
# 테스트
fn4_search_customer(customer_list)

이름d
d는 없음


## 5. 내보내기

In [104]:
# 5. 내보내기(CSV)
def fn5_save_customer_csv(customer_list):
    '매개변수로 받은 customer_list를 딕셔너리리스트로 변환해서 csv 출력'
    import csv
    if customer_list:        
        customer_dic_list = []
        for customer in customer_list:
            customer_dic_list.append(customer.as_dic())
        fieldnames = list(customer_dic_list[0].keys())
        filename = input('저장할 csv 파일명은?')
        with open('data/{}.csv'.format(filename), 'w', encoding='UTF-8', newline='') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader() # 해더 
            dict_writer.writerows(customer_dic_list)
            print(f"data/{filename}.csv 내보내기 완료")
    else:
        print('입력된 회원 데이터가 없어 csv 내보내기 취소합니다')

In [105]:
# 테스트
fn5_save_customer_csv(customer_list)

저장할 csv 파일명은?ddddd
data/ddddd.csv 내보내기 완료


## 9. 종료

In [106]:
# 9. 종료 (종료하기 전 customer_list를 txt파일에 저장하고 종료)
def fn9_save_customer_txt(customer_list):
    """customer_list를 
    to_list_style()를 이용해서 ['홍길동, 010-9999-9999, a@a.com, 30, 3, 까칠해\n',~]
    ch09_customers.txt로 백업
    """
    customer_txt_list = []
    for customer in customer_list:
        customer_txt_list.append(customer.to_list_style() + '\n')
    with open('data/ch09_customers.txt', 'w', encoding='utf-8') as f:
        f.writelines(customer_txt_list)

In [107]:
# 9번 기능 테스트
fn9_save_customer_txt(customer_list)

## main()

In [108]:
def main():
    global customer_list
    customer_list = load_customers()
    while True:
        print("1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료", end='')
        fn = input("메뉴 선택 : ")
        if fn=='1':
            # print("fn1_호출한 결과를 customer에 받아 customer_list에 append")
            customer = fn1_insert_customer_info()
            customer_list.append(customer)
        elif fn=='2':
            # print('fn2_호출')
            fn2_print_customers(customer_list)
        elif fn=='3':
            # print('fn3_호출')
            fn3_delete_customer(customer_list)
        elif fn=='4':
            # print('fn4_호출')
            fn4_search_customer(customer_list)
        elif fn=='5':
            # print('fn5_호출')
            fn5_save_customer_csv(customer_list)
        elif fn=='9':
            # print('fn9_호출')
            fn9_save_customer_txt(customer_list)
            break

In [109]:
if __name__=='__main__':
    main()

1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료메뉴 선택 : q
1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료메뉴 선택 : 1
이름 : 1
전화번호 : 1
이메일 : 2
나이 : 3
고객등급(1~5) : 4
기타 정보 : 5
1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료메뉴 선택 : 6
1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료메뉴 선택 : 6
1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료메뉴 선택 : 7
1:입력|2:전체출력|3:삭제|4:이름검색|5:csv내보내기|9: 종료

KeyboardInterrupt: Interrupted by user